In [2]:
from os.path import join as pjoin
import numpy as np
import pandas as pd
from datetime import datetime
from pytz import timezone
import pytz

In [12]:
input_filepath = "../data/raw"
train_df = pd.read_csv(input_filepath + "/train.csv")
building_df = pd.read_csv(input_filepath + "/building_metadata.csv")
train_df = train_df.merge(building_df, on = "building_id", how = "left")

In [13]:
site_info = pd.read_csv('../data/site_info.csv', delimiter = ";")
train_df = train_df.merge(site_info, on = "site_id", how = "left")

In [14]:
def reduce_mem_usage(df, verbose=True):
    """
    Takes an dataframe as argument and adjusts the datatypes of the respective
    columns to reduce memory allocation
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if (c_min > np.iinfo(np.int8).min and
                        c_max < np.iinfo(np.int8).max):
                    df[col] = df[col].astype(np.int8)
                elif (c_min > np.iinfo(np.int16).min and
                      c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min and
                      c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min and
                      c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min and
                        c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min and
                      c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    reduced_mem = 100 * (start_mem - end_mem) / start_mem
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
              .format(end_mem, reduced_mem))
    return df


In [15]:
train_df = reduce_mem_usage(train_df)

Mem. usage decreased to 1233.89 Mb (38.5% reduction)


In [29]:
#t = datetime(train_df.timestamp[1])
float(train_df.timestamp[1])


ValueError: could not convert string to float: '2016-01-01 00:00:00'

In [35]:
tz = timezone(train_df.timezone[1])

In [43]:
train_df.timestamp[1]

'2016-01-01 00:00:00'

In [44]:
t = pd.to_datetime(train_df.timestamp[1])

In [40]:
tz.localize(datetime(2002, 10, 27, 6, 0, 0))

datetime.datetime(2002, 10, 27, 6, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [46]:
tz.localize(t)

Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern')

In [25]:
datetime(2002, 10, 27, 6, 0, 0)


datetime.datetime(2002, 10, 27, 6, 0)